<a href="https://colab.research.google.com/github/Huni1i1/Running-Posture-Correction-AI/blob/main/VideoPose3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset

In [1]:
# Setup for Youtube download:
# Install pytubefix
!pip install pytubefix

import os
from pytubefix import YouTube
from pytubefix.cli import on_progress

# Make download directory
os.makedirs('/content/data', exist_ok = True)
os.makedirs('/content/data/videos', exist_ok = True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 3.2 MB/s eta 0:00:00


In [3]:
# Setup video url and download directory
video_url = input('유튜브 영상의 URL을 입력하세요: ')
# ex) https://www.youtube.com/watch?v=pDolMByXd60
video_path = '/content/data/videos'

yt = YouTube(video_url, on_progress_callback = on_progress)

ys = yt.streams.get_highest_resolution()
ys.download(video_path)

# Rename the video
video_name = input('영상의 이름을 입력하세요: ')
os.rename(video_path + '/' + yt.title + '.mp4', video_path + '/' + video_name + '.mp4')

유튜브 영상의 URL을 입력하세요: https://www.youtube.com/watch?v=pDolMByXd60
영상의 이름을 입력하세요: video1


In [ ]:
# # Video processing:
# os.makedirs('/content/data/video_processed', exist_ok = True)
# # Cut the video
# !ffmpeg -i {video_path}/{video_name}'.mp4' \
# -ss 00:00:00 -to 00:00:10 \
# -vcodec copy -acodec copy \
# {video_path}/video_cut.mp4

# Adjust fps
# fps = int(input("fps를 입력하세요: "))
# os.makedirs('/content/data/videos_{fps}fps', exist_ok = True)
# !ffmpeg -i '/content/data/videos/{video_name}.mp4' -r {fps} '/content/data/videos_{fps}fps/{video_name}_(fps)fps.mp4'

In [4]:
# Install Detectron2
!pip3 install -q torch
!pip3 install -q torchvision
!pip3 install pyyaml
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-nxlsc0ly
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-nxlsc0ly
  Resolved https://github.com/facebookresearch/detectron2.git to commit 20a6625b339c7787aac6c5536e0e613170226483
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.7 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp3

In [5]:
# Check the version
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
torch:  2.4 ; cuda:  cu121
detectron2: 0.6


In [6]:
# Clone VideoPose3D
!git clone https://github.com/facebookresearch/VideoPose3D

Cloning into 'VideoPose3D'...
remote: Enumerating objects: 121, done.
remote: Total 121 (delta 0), reused 0 (delta 0), pack-reused 121 (from 1)
Receiving objects: 100% (121/121), 9.53 MiB | 16.15 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [7]:
# Change the infer_video_d2.py
os.remove('/content/VideoPose3D/inference/infer_video_d2.py')
!git clone https://github.com/Huni1i1/det2_infer
!mv /content/det2_infer/infer_video_d2.py /content/VideoPose3D/inference/infer_video_d2.py

Cloning into 'det2_infer'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [8]:
os.makedirs('/content/VideoPose3D/checkpoint', exist_ok = True)
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin -P /content/VideoPose3D/checkpoint

--2024-10-10 11:24:56--  https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 3.163.189.96, 3.163.189.51, 3.163.189.108, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|3.163.189.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67892577 (65M) [application/octet-stream]
Saving to: ‘/content/VideoPose3D/checkpoint/pretrained_h36m_detectron_coco.bin’

pretrained_h36m_det 100%[===================>]  64.75M  63.7MB/s    in 1.0s    

2024-10-10 11:24:57 (63.7 MB/s) - ‘/content/VideoPose3D/checkpoint/pretrained_h36m_detectron_coco.bin’ saved [67892577/67892577]



In [33]:
# Step 3: inferring 2D keypoints with Detectron2
os.makedirs('/content/data/det2_output', exist_ok = True)

os.chdir('/content/VideoPose3D/inference')
!python infer_video_d2.py \
    --cfg COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x.yaml \
    --output-dir /content/VideoPose3D/data \
    --image-ext mp4 \
    /content/data/videos

[10/10 12:03:38 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from https://dl.fbaipublicfiles.com/detectron2/COCO-Keypoints/keypoint_rcnn_R_101_FPN_3x/138363331/model_final_997cc7.pkl ...
Processing /content/data/videos/video1.mp4
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-lib

In [34]:
# Step 4: creating a custom dataset
os.chdir('/content/VideoPose3D/data')
!python prepare_data_2d_custom.py -i /content/VideoPose3D/data -o video1

Parsing 2D detections from /content/VideoPose3D/data
Processing /content/VideoPose3D/data/video1.mp4.npz
729 total frames processed
1 frames were interpolated
----------
Saving...
Done.


In [35]:
# Step 5: rendering a custom video and exporting coordinates
os.chdir('/content/VideoPose3D')
!python run.py -d custom -k video1 -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject {video_name}.mp4 --viz-action custom --viz-camera 0 --viz-video {video_path}/{video_name}.mp4 --viz-output output_{video_name}.mp4 --viz-size 6

Namespace(dataset='custom', keypoints='video1', subjects_train='S1,S5,S6,S7,S8', subjects_test='S9,S11', subjects_unlabeled='', actions='*', checkpoint='checkpoint', checkpoint_frequency=10, resume='', evaluate='pretrained_h36m_detectron_coco.bin', render=True, by_subject=False, export_training_curves=False, stride=1, epochs=60, batch_size=1024, dropout=0.25, learning_rate=0.001, lr_decay=0.95, data_augmentation=True, test_time_augmentation=True, architecture='3,3,3,3,3', causal=False, channels=1024, subset=1, downsample=1, warmup=1, no_eval=False, dense=False, disable_optimizations=False, linear_projection=False, bone_length_term=True, no_proj=False, viz_subject='video1.mp4', viz_action='custom', viz_camera=0, viz_video='/content/data/videos/video1.mp4', viz_skip=0, viz_output='output_video1.mp4', viz_export=None, viz_bitrate=3000, viz_no_ground_truth=False, viz_limit=-1, viz_downsample=1, viz_size=6)
Loading dataset...
Preparing data...
Loading 2D detections...
INFO: Receptive field: